# Phase 4: Advanced Strategy Implementation

The most profitable leveraging techniques based on historical backtests.

This notebook covers:
- **Trend Following System** — sustained trend capture with leverage
- **Momentum Trading** — high-momentum asset selection
- **Scalping Engine** — high-frequency short-term trades
- **Gap Trading** — overnight/intraday gap exploitation

---

```bash
pip install pandas numpy matplotlib scipy
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

def generate_ohlcv(n=500, mu=0.10, sigma=0.25, start=100.0):
    dt = 1/252
    returns = np.random.normal(mu * dt, sigma * np.sqrt(dt), n)
    close = start * np.exp(np.cumsum(returns))
    noise = sigma * np.sqrt(dt) * 0.5
    dates = pd.date_range('2023-01-01', periods=n, freq='B')
    return pd.DataFrame({
        'open': close * (1 + np.random.normal(0, noise, n)),
        'high': close * (1 + np.abs(np.random.normal(0, noise, n))),
        'low':  close * (1 - np.abs(np.random.normal(0, noise, n))),
        'close': close,
        'volume': np.random.lognormal(10, 1, n).astype(int)
    }, index=dates)

def sma(s, p): return s.rolling(p).mean()
def ema(s, p): return s.ewm(span=p, adjust=False).mean()
def atr(df, p=14):
    tr = pd.concat([
        df['high'] - df['low'],
        abs(df['high'] - df['close'].shift()),
        abs(df['low'] - df['close'].shift())
    ], axis=1).max(axis=1)
    return tr.rolling(p).mean()

df = generate_ohlcv(750)  # 3 years
print(f"Generated {len(df)} candles")

---
## 4.1 Trend Following System

The workhorse of leveraged trading. Historical backtests show 2,834% returns over 25 years without leverage.

Key components:
- **Trend identification**: MA crossover + ADX filter
- **Entry**: Enter on trend confirmation with ATR-based position sizing
- **Exit**: Trailing stop based on ATR
- **Leverage**: Applied proportionally to trend strength

In [ ]:
def adx(df: pd.DataFrame, period: int = 14) -> pd.Series:
    """Average Directional Index — measures trend strength (not direction)."""
    plus_dm = df['high'].diff()
    minus_dm = -df['low'].diff()
    plus_dm = plus_dm.where((plus_dm > minus_dm) & (plus_dm > 0), 0.0)
    minus_dm = minus_dm.where((minus_dm > plus_dm) & (minus_dm > 0), 0.0)

    tr = pd.concat([
        df['high'] - df['low'],
        abs(df['high'] - df['close'].shift()),
        abs(df['low'] - df['close'].shift())
    ], axis=1).max(axis=1)

    atr_val = tr.rolling(period).mean()
    plus_di = 100 * (plus_dm.rolling(period).mean() / atr_val)
    minus_di = 100 * (minus_dm.rolling(period).mean() / atr_val)
    dx = 100 * abs(plus_di - minus_di) / (plus_di + minus_di)
    return dx.rolling(period).mean()


def trend_following_strategy(
    df: pd.DataFrame,
    fast_ma: int = 20,        # try 10, 20, 50
    slow_ma: int = 50,        # try 30, 50, 100, 200
    adx_threshold: float = 20, # minimum ADX to confirm trend
    atr_stop_mult: float = 3.0,# ATR multiplier for trailing stop
    base_leverage: float = 2.0,# base leverage
    max_leverage: float = 5.0  # maximum leverage
) -> pd.DataFrame:
    result = df.copy()
    result['fast'] = ema(df['close'], fast_ma)
    result['slow'] = ema(df['close'], slow_ma)
    result['adx'] = adx(df)
    result['atr'] = atr(df)

    # Trend direction
    result['trend'] = 0
    result.loc[result['fast'] > result['slow'], 'trend'] = 1
    result.loc[result['fast'] < result['slow'], 'trend'] = -1

    # Only trade when ADX confirms trend
    result['signal'] = result['trend'].where(result['adx'] > adx_threshold, 0)

    # Dynamic leverage based on ADX strength
    adx_normalized = (result['adx'] - adx_threshold).clip(lower=0) / 30  # normalize
    result['leverage'] = (base_leverage + adx_normalized * (max_leverage - base_leverage)).clip(upper=max_leverage)
    result.loc[result['signal'] == 0, 'leverage'] = 0

    # Trailing stop
    result['stop'] = np.nan
    position = 0
    stop_price = 0
    highest = 0
    stops = []
    positions = []

    for i in range(len(result)):
        sig = result['signal'].iloc[i]
        price = result['close'].iloc[i]
        atr_val = result['atr'].iloc[i] if not np.isnan(result['atr'].iloc[i]) else 0

        if position == 0 and sig != 0:
            position = sig
            stop_price = price - sig * atr_stop_mult * atr_val
            highest = price
        elif position != 0:
            # Update trailing stop
            if position == 1:
                if price > highest:
                    highest = price
                    stop_price = max(stop_price, highest - atr_stop_mult * atr_val)
                if price <= stop_price:
                    position = 0
            elif position == -1:
                if price < highest:
                    highest = price
                    stop_price = min(stop_price, highest + atr_stop_mult * atr_val)
                if price >= stop_price:
                    position = 0

            # Also exit if trend reverses
            if sig != 0 and sig != position:
                position = 0

        positions.append(position)
        stops.append(stop_price if position != 0 else np.nan)

    result['position'] = positions
    result['trailing_stop'] = stops

    # Returns with dynamic leverage
    result['market_return'] = df['close'].pct_change()
    result['strategy_return'] = (
        pd.Series(positions).shift(1).values *
        result['leverage'].shift(1).values *
        result['market_return']
    )
    result['strategy_return_unlev'] = pd.Series(positions).shift(1).values * result['market_return']

    return result.dropna()


tf = trend_following_strategy(df)

fig, axes = plt.subplots(4, 1, figsize=(14, 14), sharex=True,
                          gridspec_kw={'height_ratios': [3, 1, 1, 2]})

# Price + MAs + trailing stop
axes[0].plot(tf.index, tf['close'], color='gray', alpha=0.5, linewidth=0.8)
axes[0].plot(tf.index, tf['fast'], color='blue', linewidth=1, label='Fast EMA')
axes[0].plot(tf.index, tf['slow'], color='red', linewidth=1, label='Slow EMA')
axes[0].plot(tf.index, tf['trailing_stop'], color='orange', linewidth=0.8, linestyle=':', label='Trailing Stop')
axes[0].set_ylabel('Price ($)')
axes[0].set_title('Trend Following with ADX Filter + Dynamic Leverage', fontsize=14)
axes[0].legend()

# ADX
axes[1].plot(tf.index, tf['adx'], color='purple')
axes[1].axhline(y=20, color='green', linestyle='--', alpha=0.5, label='ADX threshold')
axes[1].set_ylabel('ADX')
axes[1].legend()

# Leverage
axes[2].fill_between(tf.index, tf['leverage'], 0, alpha=0.3, color='orange')
axes[2].set_ylabel('Leverage')

# Cumulative returns
cum_market = (1 + tf['market_return']).cumprod()
cum_unlev = (1 + tf['strategy_return_unlev']).cumprod()
cum_lev = (1 + tf['strategy_return'].clip(-0.99, None)).cumprod()
axes[3].plot(tf.index, cum_market, color='gray', label='Buy & Hold')
axes[3].plot(tf.index, cum_unlev, color='blue', label='Strategy (no leverage)')
axes[3].plot(tf.index, cum_lev, color='green', linewidth=2, label='Strategy (dynamic leverage)')
axes[3].set_ylabel('Cumulative Return')
axes[3].legend()

plt.tight_layout()
plt.show()

print(f"Buy & Hold: {(cum_market.iloc[-1] - 1):.2%}")
print(f"Strategy (unleveraged): {(cum_unlev.iloc[-1] - 1):.2%}")
print(f"Strategy (dynamic leverage): {(cum_lev.iloc[-1] - 1):.2%}")

### Exercise 4.1

1. Remove the ADX filter (set threshold to 0). How does it affect performance in sideways markets?
2. Change `atr_stop_mult` from 3.0 to 1.5 and 5.0. Tighter stops = more trades but more whipsaws. Wider stops = fewer trades but larger drawdowns per trade.
3. Cap leverage at 2x instead of 5x. How much return do you sacrifice for lower risk?

In [ ]:
# YOUR CODE HERE


---
## 4.2 Momentum Trading

Momentum strategies select assets that have been performing well recently, betting that winners continue to win.

Key idea: Buy the top performers over the lookback period, short the bottom performers.

In [ ]:
# Generate a universe of assets
n_assets = 10
universe = {}
for i in range(n_assets):
    mu = np.random.uniform(-0.05, 0.20)
    sigma = np.random.uniform(0.15, 0.40)
    universe[f'Asset_{i+1}'] = generate_ohlcv(500, mu=mu, sigma=sigma, start=np.random.uniform(50, 200))['close']

universe_df = pd.DataFrame(universe)
universe_returns = universe_df.pct_change().dropna()

print("Universe of assets:")
for col in universe_df.columns:
    total_ret = (universe_df[col].iloc[-1] / universe_df[col].iloc[0]) - 1
    vol = universe_returns[col].std() * np.sqrt(252)
    print(f"  {col}: return={total_ret:+.1%}, vol={vol:.1%}")

In [ ]:
def momentum_strategy(
    returns_df: pd.DataFrame,
    lookback: int = 60,      # momentum lookback (try 20, 60, 120, 252)
    hold_period: int = 20,   # rebalance frequency
    top_n: int = 3,          # number of assets to go long
    bottom_n: int = 2,       # number of assets to go short
    leverage: float = 2.0
) -> pd.DataFrame:
    """Cross-sectional momentum: long winners, short losers."""
    prices = (1 + returns_df).cumprod()
    portfolio_returns = []
    holdings_history = []

    for start in range(lookback, len(returns_df) - hold_period, hold_period):
        # Rank assets by past returns
        past_returns = prices.iloc[start] / prices.iloc[start - lookback] - 1
        ranked = past_returns.sort_values(ascending=False)

        longs = ranked.index[:top_n].tolist()
        shorts = ranked.index[-bottom_n:].tolist()

        # Equal weight within longs and shorts
        for day in range(hold_period):
            idx = start + day
            if idx >= len(returns_df):
                break
            day_returns = returns_df.iloc[idx]
            long_ret = day_returns[longs].mean() if longs else 0
            short_ret = -day_returns[shorts].mean() if shorts else 0
            total = (long_ret + short_ret) * leverage
            portfolio_returns.append({
                'date': returns_df.index[idx],
                'return': total,
                'long_ret': long_ret * leverage,
                'short_ret': short_ret * leverage,
                'longs': ', '.join(longs),
                'shorts': ', '.join(shorts)
            })

    return pd.DataFrame(portfolio_returns).set_index('date')


mom = momentum_strategy(universe_returns, lookback=60, hold_period=20, leverage=2.0)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# Cumulative returns
cum = (1 + mom['return']).cumprod()
cum_long = (1 + mom['long_ret']).cumprod()
cum_short = (1 + mom['short_ret']).cumprod()
equal_weight = (1 + universe_returns.mean(axis=1)).cumprod()

ax1.plot(cum.index, cum, color='green', linewidth=2, label='Momentum (L/S, 2x lev)')
ax1.plot(cum_long.index, cum_long, color='blue', alpha=0.5, label='Long only')
ax1.plot(cum_short.index, cum_short, color='red', alpha=0.5, label='Short only')
ax1.plot(equal_weight.index[equal_weight.index.isin(cum.index)],
         equal_weight[equal_weight.index.isin(cum.index)],
         color='gray', linestyle='--', label='Equal weight (no strategy)')
ax1.set_ylabel('Cumulative Return')
ax1.set_title('Momentum Strategy: Long Winners, Short Losers', fontsize=14)
ax1.legend()

# Rolling 60-day return
rolling = mom['return'].rolling(60).sum()
ax2.fill_between(rolling.index, rolling * 100, 0,
                  where=rolling >= 0, color='green', alpha=0.3)
ax2.fill_between(rolling.index, rolling * 100, 0,
                  where=rolling < 0, color='red', alpha=0.3)
ax2.set_ylabel('Rolling 60d Return (%)')
ax2.set_xlabel('Date')

plt.tight_layout()
plt.show()

print(f"Total return: {(cum.iloc[-1] - 1):.2%}")
print(f"Ann. Sharpe: {mom['return'].mean() / mom['return'].std() * np.sqrt(252):.2f}")

### Exercise 4.2

1. Change the lookback to 20 (short-term) and 252 (annual). How does the performance change?
2. What happens with long-only momentum (set `bottom_n=0`)? Is the short side adding or losing value?
3. Add a "momentum crash" filter: reduce leverage when the strategy has lost >5% in the past 20 days.

In [ ]:
# YOUR CODE HERE


---
## 4.3 Scalping Engine

High-frequency, short-term trades capturing small price movements. Uses leverage to amplify small moves.

Key: low latency, tight spreads, and strict risk management.

In [ ]:
def parabolic_sar(df: pd.DataFrame, af_start=0.02, af_increment=0.02, af_max=0.20):
    """Parabolic SAR indicator."""
    high = df['high'].values
    low = df['low'].values
    close = df['close'].values
    n = len(close)

    sar = np.zeros(n)
    trend = np.ones(n)  # 1 = up, -1 = down
    af = af_start
    ep = high[0]
    sar[0] = low[0]

    for i in range(1, n):
        if trend[i-1] == 1:  # uptrend
            sar[i] = sar[i-1] + af * (ep - sar[i-1])
            sar[i] = min(sar[i], low[i-1], low[i-2] if i >= 2 else low[i-1])

            if low[i] < sar[i]:  # reversal
                trend[i] = -1
                sar[i] = ep
                ep = low[i]
                af = af_start
            else:
                trend[i] = 1
                if high[i] > ep:
                    ep = high[i]
                    af = min(af + af_increment, af_max)
        else:  # downtrend
            sar[i] = sar[i-1] + af * (ep - sar[i-1])
            sar[i] = max(sar[i], high[i-1], high[i-2] if i >= 2 else high[i-1])

            if high[i] > sar[i]:  # reversal
                trend[i] = 1
                sar[i] = ep
                ep = high[i]
                af = af_start
            else:
                trend[i] = -1
                if low[i] < ep:
                    ep = low[i]
                    af = min(af + af_increment, af_max)

    return pd.Series(sar, index=df.index), pd.Series(trend, index=df.index)


def scalping_strategy(
    df: pd.DataFrame,
    leverage: float = 10.0,      # scalping uses high leverage
    take_profit_atr: float = 1.0,# quick TP
    stop_loss_atr: float = 0.5,  # tight SL
    fee_bps: float = 5.0         # critical for scalping!
) -> pd.DataFrame:
    result = df.copy()
    sar, trend = parabolic_sar(df)
    result['sar'] = sar
    result['trend'] = trend
    result['atr'] = atr(df, 10)  # shorter ATR for scalping

    trades = []
    position = 0
    entry_price = 0

    for i in range(1, len(result)):
        price = result['close'].iloc[i]
        atr_val = result['atr'].iloc[i]
        if np.isnan(atr_val):
            continue

        # Check exits
        if position != 0:
            pnl_points = position * (price - entry_price)
            if pnl_points >= take_profit_atr * atr_val:  # TP
                fee = 2 * fee_bps / 10000  # entry + exit
                ret = (position * (price - entry_price) / entry_price - fee) * leverage
                trades.append({'return': ret, 'exit': 'TP', 'date': result.index[i]})
                position = 0
            elif pnl_points <= -stop_loss_atr * atr_val:  # SL
                fee = 2 * fee_bps / 10000
                ret = (position * (price - entry_price) / entry_price - fee) * leverage
                trades.append({'return': ret, 'exit': 'SL', 'date': result.index[i]})
                position = 0

        # Check entries (SAR reversal)
        if position == 0:
            if result['trend'].iloc[i] == 1 and result['trend'].iloc[i-1] == -1:
                position = 1
                entry_price = price
            elif result['trend'].iloc[i] == -1 and result['trend'].iloc[i-1] == 1:
                position = -1
                entry_price = price

    return result, pd.DataFrame(trades)


scalp_result, scalp_trades = scalping_strategy(df, leverage=10, fee_bps=5)

# Visualize a window
window = slice(100, 200)
sr = scalp_result.iloc[window]

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(sr.index, sr['close'], color='steelblue', linewidth=1)
up = sr[sr['trend'] == 1]
down = sr[sr['trend'] == -1]
ax.scatter(up.index, up['sar'], color='green', s=10, label='SAR (up)')
ax.scatter(down.index, down['sar'], color='red', s=10, label='SAR (down)')
ax.set_title('Parabolic SAR Scalping (zoomed window)', fontsize=14)
ax.set_ylabel('Price ($)')
ax.legend()
plt.tight_layout()
plt.show()

if len(scalp_trades) > 0:
    print(f"Total trades: {len(scalp_trades)}")
    print(f"Win rate: {(scalp_trades['return'] > 0).mean():.1%}")
    print(f"Avg return/trade: {scalp_trades['return'].mean():.2%}")
    print(f"Total return: {(1 + scalp_trades['return']).prod() - 1:.2%}")
    print(f"Exit breakdown: {scalp_trades['exit'].value_counts().to_dict()}")
    print(f"\nNote: At 10x leverage with 5bps fees, each round trip costs {2*5*10/100:.1f}% of leveraged position")
else:
    print("No trades generated — try adjusting parameters.")

### Exercise 4.3

1. Increase `fee_bps` to 20 (expensive exchange). Is scalping still profitable?
2. Adjust `take_profit_atr` and `stop_loss_atr` ratios. What reward:risk ratio works best?
3. Why is fee sensitivity the #1 concern for scalping strategies?

In [ ]:
# YOUR CODE HERE


---
## 4.4 Gap Trading

Exploit the gap between the previous day's close and the next day's open.

In [ ]:
def gap_trading_strategy(
    df: pd.DataFrame,
    min_gap_pct: float = 0.005, # minimum gap to trade (0.5%)
    strategy: str = 'fade',     # 'fade' (mean reversion) or 'follow' (momentum)
    leverage: float = 5.0,
    hold_until: str = 'close'   # 'close' = exit at end of day, 'fill' = exit when gap fills
) -> pd.DataFrame:
    """Trade opening gaps."""
    result = df.copy()
    result['gap_pct'] = (df['open'] - df['close'].shift(1)) / df['close'].shift(1)

    trades = []
    for i in range(1, len(result)):
        gap = result['gap_pct'].iloc[i]
        if abs(gap) < min_gap_pct:
            continue

        open_price = result['open'].iloc[i]
        close_price = result['close'].iloc[i]
        prev_close = result['close'].iloc[i-1]

        if strategy == 'fade':
            # Fade the gap: short gap-ups, long gap-downs (bet gap will fill)
            direction = -1 if gap > 0 else 1
        else:
            # Follow the gap: long gap-ups, short gap-downs
            direction = 1 if gap > 0 else -1

        # Exit at close of the day
        pnl = direction * (close_price - open_price) / open_price * leverage

        # Check if gap was filled during the day
        gap_filled = False
        if gap > 0 and result['low'].iloc[i] <= prev_close:
            gap_filled = True
        elif gap < 0 and result['high'].iloc[i] >= prev_close:
            gap_filled = True

        trades.append({
            'date': result.index[i],
            'gap_pct': gap,
            'direction': 'long' if direction == 1 else 'short',
            'return': pnl,
            'gap_filled': gap_filled
        })

    return pd.DataFrame(trades).set_index('date') if trades else pd.DataFrame()


# Compare fade vs follow
fade_trades = gap_trading_strategy(df, strategy='fade', leverage=5)
follow_trades = gap_trading_strategy(df, strategy='follow', leverage=5)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, trades, name in [(axes[0], fade_trades, 'Fade Gaps'), (axes[1], follow_trades, 'Follow Gaps')]:
    if len(trades) > 0:
        cum = (1 + trades['return']).cumprod()
        ax.plot(cum.index, cum, color='steelblue', linewidth=1.5)
        ax.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5)
        ax.set_title(f'{name} (5x leverage)', fontsize=13)
        ax.set_ylabel('Cumulative Return')

        win_rate = (trades['return'] > 0).mean()
        total_ret = cum.iloc[-1] - 1
        fill_rate = trades['gap_filled'].mean()
        ax.text(0.05, 0.95, f'Trades: {len(trades)}\nWin: {win_rate:.0%}\nReturn: {total_ret:.1%}\nGap fill: {fill_rate:.0%}',
                transform=ax.transAxes, verticalalignment='top', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

if len(fade_trades) > 0:
    print(f"Gap fill rate: {fade_trades['gap_filled'].mean():.1%} of gaps filled within the day")
    print(f"Average gap size: {fade_trades['gap_pct'].abs().mean():.2%}")

### Exercise 4.4

1. Change `min_gap_pct` to filter only large gaps (>1%). Does the strategy improve?
2. On the fade strategy, what's the relationship between gap fill rate and profitability?
3. Combine gap trading with volume: only take trades when the gap day's volume is above average. Does it help?

In [ ]:
# YOUR CODE HERE


---
## 4.5 Comprehension Check

1. Why does the ADX filter improve trend following? What does ADX actually measure?
2. Momentum strategies often suffer "momentum crashes" (sudden reversals). What causes these and how would you protect against them?
3. At 10x leverage, a 0.1% move = 1% on your equity. If fees are 10 bps per trade (20 bps round trip), how much does the price need to move before you break even?
4. Why do most gap fade strategies work better than gap follow strategies? Think about market microstructure.
5. Which of these four strategies would you use for: (a) trending BTC, (b) range-bound S&P 500, (c) volatile penny stock?

In [ ]:
# YOUR ANSWERS HERE
